In [1]:

import re
import json
from bs4 import BeautifulSoup, NavigableString,Tag
from tabulate import tabulate
import unicodedata
import time
import requests
from markdownify import markdownify as md
import base64
from urllib.parse import urlparse, parse_qs
import uuid

test ccpl mới

In [2]:
# format các dạng bảng
def table_to_markdown(table):
    if table is None:
        return ""
    rows = []
    headers = []

    for row in table.find_all('tr'):
        row_data = []
        for cell in row.find_all(['th', 'td']):
            cell_text = cell.get_text(strip=True)
            row_data.append(cell_text)
        if not headers:
            headers = row_data
        else:
            rows.append(row_data)

    return tabulate(rows, headers, tablefmt='pipe', disable_numparse=True)

In [3]:
############ Điều 1, 2, 3 + replace
import re

def split_by_dieu_new1(text):
    # def extract_dieu_positions(text):
    if text is None:
        return[text]

        
    dieu_positions = [(match.start(), match.end()) for match in re.finditer(r'Điều\s+\d+[a-zA-Z0-9đĐ]*(?:,\s*\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)*(?:\s+và\s+\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)?', text, re.IGNORECASE)]
    if not dieu_positions:
        return [text]

    split_text = []
    current_position = 0

    for start, end in dieu_positions:
        if current_position < start:
            split_text.append(text[current_position:start].strip())
        split_text.append(text[start:end].strip())
        current_position = end

    if current_position < len(text):
        split_text.append(text[current_position:].strip())

    return split_text
################
def extract_dieu_err(text):
    # Define the pattern to match "Điều" followed by numbers and possibly letters or other characters
    pattern = r'Điều\s+\d+[a-zA-Z0-9đĐ]*(?:,\s*\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)*(?:\s+và\s+\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)?'

    # Find all matches of the pattern in the text
    matches = list(re.finditer(pattern, text, re.IGNORECASE))
    matched_phrases = [match.group(0) for match in matches]
    check_phrases = []

    # Loop through each phrase in matched_phrases
    for phrase in matched_phrases:
        if ',' in phrase or ' và ' in phrase:
            check_phrases.append(phrase)
    return check_phrases
##############
def process_dieu_parts(text):
    dieu_final = []
    for match in text:
        if ',' in match or ' và ' in match:
            splitted = re.split(r',| và ', match)
            for temp in splitted:
                if re.match(r'(?i)Điều\s+\d+', temp):
                    dieu_final.append(temp)
                else:
                    dieu_final.append('Điều ' + temp)

    return dieu_final
###############
def replace_dieu_in_text(text):
    if text is None:
        return[text]
    split_text = split_by_dieu_new1(text)
    matches = extract_dieu_err(text)
    if not matches:
        return text
    dieu_final = process_dieu_parts(matches)
    replaced_text = []
    for part in split_text:
        if re.match(r'(?i)Điều\s+\d+', part) and " và " in part:
            for dieu in dieu_final:
                replaced_text.append(dieu)
        else:
            replaced_text.append(part)

    return ' '.join(replaced_text)

#############
#############
########### khoản 1, 2,3  và .... + replace


import re

def split_by_khoan_new1(text):
    if text is None:
        return[text]       
    khoan_positions = [(match.start(), match.end()) for match in re.finditer(r'khoản\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?(?:,\s*| và |, và |))*[a-zA-Z0-9đĐ]+(?:\.\d+)?)(?=\s|,|\.|;|$)', text, re.IGNORECASE)]
    if not khoan_positions:
        return [text]

    split_text = []
    current_position = 0

    for start, end in khoan_positions:
        if current_position < start:
            split_text.append(text[current_position:start].strip())
        split_text.append(text[start:end].strip())
        current_position = end

    if current_position < len(text):
        split_text.append(text[current_position:].strip())

    return split_text

#####################
def extract_khoan_err(text):
    # Define the pattern to match "khoản" followed by numbers and possibly letters or other characters
    pattern = r'khoản\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?(?:,\s*| và |, và |))*[a-zA-Z0-9đĐ]+(?:\.\d+)?)(?=\s|,|\.|;|$)'

    # Find all matches of the pattern in the text
    matches = list(re.finditer(pattern, text, re.IGNORECASE))
    matched_phrases = [match.group(0) for match in matches]
    check_phrases = []

    # Loop through each phrase in matched_phrases
    for phrase in matched_phrases:
        if ',' in phrase or ' và ' in phrase:
            check_phrases.append(phrase)
    return check_phrases
################
def process_khoan_parts(text):
    khoan_final = []
    for match in text:
        if ',' in match or ' và ' in match:
            splitted = re.split(r',| và ', match)
            for temp in splitted:
                if re.match(r'(?i)khoản\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?)+)', temp):
                    khoan_final.append(temp)
                else:
                    khoan_final.append('khoản ' + temp)

    return khoan_final
###############

def replace_khoan_in_text(text):
    if text is None:
        return[text]
    split_text = split_by_khoan_new1(text)
    matches = extract_khoan_err(text)
    if not matches:
        return text
    khoan_final = process_khoan_parts(matches)
    replaced_text = []
    for part in split_text:
        if re.match(r'(?i)khoản\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?)+)', part) and " và " in part:
            for khoan in khoan_final:
                replaced_text.append(khoan)
        else:
            replaced_text.append(part)

    return ' '.join(replaced_text)
#############
#############
################# Điểm 1,2,3 và .... + replace
import re

def split_by_diem_new1(text):
    def extract_diem_positions(text):
        return [(match.start(), match.end()) for match in re.finditer(r'điểm\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?(?:,\s+| và |, và |))*[a-zA-Z0-9đĐ]+(?:\.\d+)?)(?=\s|,|\.|;|$|\s+và\s+[a-zA-ZđĐ0-9])', text, re.IGNORECASE)]
        
    diem_positions = extract_diem_positions(text)
    if not diem_positions:
        return [text]

    split_text = []
    current_position = 0

    for start, end in diem_positions:
        if current_position < start:
            split_text.append(text[current_position:start].strip())
        split_text.append(text[start:end].strip())
        current_position = end

    if current_position < len(text):
        split_text.append(text[current_position:].strip())

    return split_text
###########################
def extract_diem_err(text):
    # Define the pattern to match "điểm" followed by numbers and possibly letters or other characters
    pattern = r'điểm\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?(?:,\s+| và |, và |))*[a-zA-Z0-9đĐ]+(?:\.\d+)?)(?=\s|,|\.|;|$|\s+và\s+[a-zA-ZđĐ0-9])'

    # Find all matches of the pattern in the text
    matches = list(re.finditer(pattern, text, re.IGNORECASE))
    matched_phrases = [match.group(0) for match in matches]
    check_phrases = []

    # Loop through each phrase in matched_phrases
    for phrase in matched_phrases:
        if ',' in phrase or ' và ' in phrase:
            check_phrases.append(phrase)
    return check_phrases
#########################
def process_diem_parts(text):
    diem_final = []
    for match in text:
        if ',' in match or ' và ' in match:
            splitted = re.split(r',| và ', match)
            for temp in splitted:
                if re.match(r'(?i)điểm\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?)+)', temp):
                    diem_final.append(temp)
                else:
                    diem_final.append('điểm ' + temp)

    return diem_final
#########################
def replace_diem_in_text(text):
    if text is None:
        return[text]
    split_text = split_by_diem_new1(text)
    matches = extract_diem_err(text)
    if not matches:
        return text
    diem_final = process_diem_parts(matches)
    replaced_text = []
    for part in split_text:
        if re.match(r'(?i)điểm\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?)+)', part) and " và " in part:
            for diem in diem_final:
                replaced_text.append(diem)
        else:
            replaced_text.append(part)

    return ' '.join(replaced_text)


In [4]:
def split_and_transform_dieu_ranges(text):
    if text is None:
        return [text]

    dieu_positions = [(match.start(), match.end()) for match in re.finditer(r'Điều\s+\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', text, re.IGNORECASE)]

    if not dieu_positions:
        return [text]

    new_split_text = []
    current_position = 0
    start_dieu = None

    for i, (start, end) in enumerate(dieu_positions):
        if current_position < start:
            split_text_part = text[current_position:end].strip()
            if "từ Điều" in split_text_part:
                start_dieu_match = re.search(r'Điều\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE)
                if start_dieu_match:
                    start_dieu = start_dieu_match.group(1)
            elif "đến Điều" in split_text_part or "tới Điều" in split_text_part:
                end_dieu_match = re.search(r'Điều\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE)
                if end_dieu_match:
                    end_dieu = end_dieu_match.group(1)
                    start_index = int(re.sub("[a-zA-ZđĐ]", "", start_dieu)) if start_dieu else None
                    end_index = int(re.sub("[a-zA-ZđĐ]", "", end_dieu)) if end_dieu else None
                    if start_index is not None and end_index is not None:
                        for d in range(start_index, end_index + 1):
                            new_split_text.append(f"Điều {d}")
            else:
                new_split_text.append(split_text_part)

        current_position = end

    if current_position < len(text):
        split_text_part = text[current_position:].strip()
        if "từ Điều" in split_text_part:
            start_dieu_match = re.search(r'Điều\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE)
            if start_dieu_match:
                start_dieu = start_dieu_match.group(1)
        elif "đến Điều" in split_text_part or "tới Điều" in split_text_part:
            end_dieu_match = re.search(r'Điều\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE)
            if end_dieu_match:
                end_dieu = end_dieu_match.group(1)
                start_index = int(re.sub("[a-zA-ZđĐ]", "", start_dieu)) if start_dieu else None
                end_index = int(re.sub("[a-zA-ZđĐ]", "", end_dieu)) if end_dieu else None
                if start_index is not None and end_index is not None:
                    for d in range(start_index, end_index + 1):
                        new_split_text.append(f"Điều {d}")
        else:
            new_split_text.append(split_text_part)

    return new_split_text

def replace_dieu_ranges(text):
    # Tìm khoảng "từ Điều X đến Điều Y" hoặc "từ Điều X tới Điều Y"
    if text is None:
        return[text]
    dieu_range_match = re.search(r'từ Điều \d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)* (?:đến|tới) Điều \d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', text, re.IGNORECASE)
    if not dieu_range_match:
        return text

    dieu_range_text = dieu_range_match.group(0)
    dieu_list = split_and_transform_dieu_ranges(dieu_range_text)
    
    # Tạo chuỗi mới từ danh sách các Điều
    dieu_list_str = ', '.join(dieu_list)

    # Thay thế đoạn văn bản "từ Điều X đến Điều Y" hoặc "từ Điều X tới Điều Y" bằng danh sách các Điều
    new_text = text.replace(dieu_range_text, dieu_list_str)

    return new_text

def split_and_transform_khoan_ranges(text):
    if text is None:
        return [text]
        

    khoan_positions = [(match.start(), match.end()) for match in re.finditer(r'khoản\s+\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', text, re.IGNORECASE)]
    if not khoan_positions:
        return [text]

    new_split_text = []
    current_position = 0
    start_khoan = None

    for i, (start, end) in enumerate(khoan_positions):
        if current_position < start:
            split_text_part = text[current_position:end].strip()
            if "từ khoản" in split_text_part.lower():
                start_khoan = re.search(r'khoản\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
            elif "đến khoản" in split_text_part.lower() or "tới khoản" in split_text_part.lower():
                end_khoan = re.search(r'khoản\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
                start_index = int(re.sub("[a-zA-ZđĐ]", "", start_khoan))
                end_index = int(re.sub("[a-zA-ZđĐ]", "", end_khoan))
                for d in range(start_index, end_index + 1):
                    new_split_text.append(f"khoản {d}")
            else:
                new_split_text.append(split_text_part)
        current_position = end

    if current_position < len(text):
        split_text_part = text[current_position:].strip()
        if "từ khoản" in split_text_part.lower():
            start_khoan = re.search(r'khoản\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
        elif "đến khoản" in split_text_part.lower() or "tới khoản" in split_text_part.lower():
            end_khoan = re.search(r'khoản\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
            start_index = int(re.sub("[a-zA-ZđĐ]", "", start_khoan))
            end_index = int(re.sub("[a-zA-ZđĐ]", "", end_khoan))
            for d in range(start_index, end_index + 1):
                new_split_text.append(f"khoản {d}")
        else:
            new_split_text.append(split_text_part)

    return new_split_text

def replace_khoan_ranges(text):
    # Tìm khoảng "từ khoản X đến khoản Y" hoặc "từ khoản X tới khoản Y"
    if text is None:
        return[text]
    khoan_range_matches = re.finditer(r'từ khoản \d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)* (?:đến|tới) khoản \d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', text, re.IGNORECASE)
    
    new_text = text
    offset = 0
    
    for match in khoan_range_matches:
        khoan_range_text = match.group(0)
        khoan_list = split_and_transform_khoan_ranges(khoan_range_text)
        
        # Tạo chuỗi mới từ danh sách các khoản
        khoan_list_str = ', '.join(khoan_list)

        # Tính toán vị trí mới sau khi thay thế
        start, end = match.start() + offset, match.end() + offset
        new_text = new_text[:start] + khoan_list_str + new_text[end:]
        
        # Cập nhật offset
        offset += len(khoan_list_str) - len(khoan_range_text)

    return new_text

def split_and_transform_diem_ranges(text):
    if text is None:
        return [text]
        

    diem_positions = [(match.start(), match.end()) for match in re.finditer(r'điểm\s+\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', text, re.IGNORECASE)]
    if not diem_positions:
        return [text]

    new_split_text = []
    current_position = 0
    start_diem = None

    for i, (start, end) in enumerate(diem_positions):
        if current_position < start:
            split_text_part = text[current_position:end].strip()
            if "từ điểm" in split_text_part.lower():
                start_diem = re.search(r'điểm\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
            elif "đến điểm" in split_text_part.lower() or "tới điểm" in split_text_part.lower():
                end_diem = re.search(r'điểm\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
                start_index = int(re.sub("[a-zA-ZđĐ]", "", start_diem))
                end_index = int(re.sub("[a-zA-ZđĐ]", "", end_diem))
                for d in range(start_index, end_index + 1):
                    new_split_text.append(f"điểm {d}")
            else:
                new_split_text.append(split_text_part)
        current_position = end

    if current_position < len(text):
        split_text_part = text[current_position:].strip()
        if "từ điểm" in split_text_part.lower():
            start_diem = re.search(r'điểm\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
        elif "đến điểm" in split_text_part.lower() or "tới điểm" in split_text_part.lower():
            end_diem = re.search(r'điểm\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
            start_index = int(re.sub("[a-zA-ZđĐ]", "", start_diem))
            end_index = int(re.sub("[a-zA-ZđĐ]", "", end_diem))
            for d in range(start_index, end_index + 1):
                new_split_text.append(f"điểm {d}")
        else:
            new_split_text.append(split_text_part)

    return new_split_text

def replace_diem_ranges(text):
    # Tìm khoảng "từ điểm X đến điểm Y" hoặc "từ điểm X tới điểm Y"
    if text is None:
        return[text]
    diem_range_matches = re.finditer(r'từ điểm \d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)* (?:đến|tới) điểm \d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', text, re.IGNORECASE)
    
    new_text = text
    offset = 0
    
    for match in diem_range_matches:
        diem_range_text = match.group(0)
        diem_list = split_and_transform_diem_ranges(diem_range_text)
        
        # Tạo chuỗi mới từ danh sách các điểm
        diem_list_str = ', '.join(diem_list)

        # Tính toán vị trí mới sau khi thay thế
        start, end = match.start() + offset, match.end() + offset
        new_text = new_text[:start] + diem_list_str + new_text[end:]
        
        # Cập nhật offset
        offset += len(diem_list_str) - len(diem_range_text)

    return new_text

In [5]:
############## dạng ..... 1, 2, 3 và .....,......
def processing_ccpl_full(text):
    if text is None:
        return [text]  
    check = replace_dieu_ranges(replace_khoan_ranges(replace_diem_ranges(text)))
    check2 = replace_dieu_in_text(replace_khoan_in_text(replace_diem_in_text(check)))

    return check2

In [6]:
########### Cắt từ điều này tới điều kia


def split_by_dieu(text):
    if text is None: 
        return[text]
    
    text = ' ' + text
    dieu_positions = [(match.start(), match.end()) for match in re.finditer(r'Điều\s+\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', text, re.IGNORECASE)]
    if not dieu_positions:
        return [text]

    split_text = []
    current_position = 0
    
    for i, (start, end) in enumerate(dieu_positions):
        if current_position < start:
            split_text.append(text[current_position:end].strip())
        current_position = end

    if current_position < len(text):
        split_text.append(text[current_position:].strip())

    return split_text


In [7]:
########### Cắt từ khoản này tới khoản kia

def split_by_khoan(text):
    if text is None:
            return[text]

    text = ' ' + text
    khoan_positions = [(match.start(), match.end()) for match in re.finditer(r'khoản\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?(?:,\s*| và |, và |))*[a-zA-Z0-9đĐ]+(?:\.\d+)?)(?=\s|,|\.|;|$)', text, re.IGNORECASE)]
    if not khoan_positions:
        return [text]

    split_text = []
    current_position = 0

    first_khoan_start = khoan_positions[0][0]
    if first_khoan_start > 0:
        split_text.append(text[:first_khoan_start].strip())

    for i, (start, end) in enumerate(khoan_positions):
        if current_position < start:
            split_text.append(text[current_position:end].strip())
        current_position = end

    if current_position < len(text):
        split_text.append(text[current_position:].strip())

    return split_text

In [8]:
# Hàm trích xuất điều
def extract_dieu(text):
    # Find individual "Điều" references
    dieu_list = re.findall(r'Điều\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', text, re.IGNORECASE)

    # Find joined "Điều" lists
    joined_dieu_list = re.findall(r'Điều\s+\d+[a-zA-Z0-9đĐ]*(?:,\s*\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)*(?:\s+và\s+\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)?', text, re.IGNORECASE)

    # Process joined "Điều" lists
    for item in joined_dieu_list:
        # Find all "Điều" references in the joined list
        joined_dieu_refs = re.findall(r'\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', item)
        # Add only the unique "Điều" references to the main list
        for ref in joined_dieu_refs:
            if ref not in dieu_list:
                dieu_list.append(ref)
    return dieu_list

# Hàm trích xuất khoản
def extract_khoan(text):
    pattern = r'khoản\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?(?:,\s*| và |, và |))*[a-zA-Z0-9đĐ]+(?:\.\d+)?)(?=\s|,|\.|;|$)'
    matches = re.findall(pattern, text, re.IGNORECASE)
    final_khoan_list = []
    for match in matches:
        parts = re.split(r',\s*|\s+và\s+', match)
        for part in parts:
            part = part.strip().strip('.').lower()
            if '-' in part:
                start, end = part.split('-')
                for idx in range(int(start), int(end) + 1):
                    final_khoan_list.append(str(idx))
            elif len(part) == 1 and part.isalpha() or part[0].isdigit():
                final_khoan_list.append(part)
    return final_khoan_list

# Hàm trích xuất điểm
def extract_diem(text):
    pattern = r'điểm\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?(?:,\s+| và |, và |))*[a-zA-Z0-9đĐ]+(?:\.\d+)?)(?=\s|,|\.|;|$|\s+và\s+[a-zA-ZđĐ0-9])'
    matches = re.findall(pattern, text, re.IGNORECASE)
    final_diem_list = []
    for match in matches:
        parts = re.split(r',\s*|\s+và\s+', match)
        for part in parts:
            part = part.strip().strip('.').lower()
            if re.match(r'^[a-zA-ZđĐ]\d*(\.\d+)*$', part) or re.match(r'^\d+(\.\d+)*$', part):
                final_diem_list.append(part)
    return final_diem_list

# hàm xử lý đầu ra ccpl
def processing_output_ccpl(input_data):
    output_array = []
    for dieu in input_data:
        dieu_value = ", ".join(dieu["Dieu"])
        if dieu["Khoan"] == [[]]:
            output_array.extend([
                    dieu_value,
                    "0",
                    "0"
                ])
        elif dieu["Khoan"]:
            for khoan in dieu["Khoan"]:
                for khoan_item in khoan["Khoan"]:
                    if khoan["Diem"] == [[]]:
                        output_array.extend([
                            dieu_value,
                            khoan_item,
                            "0"
                        ])
                    elif khoan["Diem"]:
                        for diem in khoan["Diem"]:
                            for diem_item in diem:
                                output_array.extend([
                                    dieu_value,
                                    khoan_item,
                                    diem_item
                                ])
                                # print(output_array)
                    else:
                        output_array.extend([
                            dieu_value,
                            khoan_item,
                            "0"
                        ])
        else:
            output_array.extend([
                dieu_value,
                "0",
                "0"
            ])
    if output_array:
        return output_array

    else:
        return []

# Function to remove duplicate ccpl entries
def remove_duplicates_ccpls(data):
    for item in data:
        seen = set()
        unique_ccpls = []
        for ccpl in item['ccpls']:
            if ccpl['Dieu']:  # Kiểm tra nếu 'Dieu' không phải là rỗng
                ccpl_tuple = (ccpl['LawID'], ccpl['LawTitle'], ccpl['Dieu'], ccpl['Khoan'], ccpl['Diem'])
                if ccpl_tuple not in seen:
                    seen.add(ccpl_tuple)
                    unique_ccpls.append(ccpl)
        item['ccpls'] = unique_ccpls
    return data

In [9]:
def processing_redundant_data(data):
    soup = BeautifulSoup(data, 'html.parser')

    # Xóa dẫn link dạng bảng có chữ tải về
    for p_tag in soup.find_all('table'):
        try:
            a_tag = p_tag.find('a')
            # Tìm thẻ <img> bên trong thẻ <a>
            img_tag = a_tag.find('img')
            # Kiểm tra href và src có tồn tại không trước khi kiểm tra endswith
            if (a_tag.get('href') and not a_tag.get('href').endswith(('.doc', '.docx', '.xls', '.xlsx', '.zip', '.rar', '.jpg', '.pdf', '.png', '.jpeg'))) and (img_tag and img_tag.get('src') and img_tag.get('src').endswith(('png', 'jpg', 'jpeg'))): 
                # Xử lý thẻ <p> nếu điều kiện thỏa mãn
                # print(p_tag.text)
                p_tag.decompose()
        except Exception as e:
            print(f"Error processing tag: {e}")
    
    # Tìm tất cả các thẻ <p>
    p_tags = soup.find_all('p')
    # Kiểm tra từng thẻ <p> để xem có chứa chuỗi "lưu ý: khi thực hiện cải cách tiền lương..." không
    for p_tag in p_tags:
        if "lưu ý: khi thực hiện cải cách tiền lương từ ngày 1/7/2024 sẽ bãi bỏ mức lương cơ sở và hệ số lương hiện nay, xây dựng bảng lương mới, theo mục 2 nghị quyết 27-nq/tw năm 2018" in p_tag.get_text().lower():
            p_tag.decompose()
        if "theo điều 3, điều 4 thông tư 29/2012/tt- bgdđt đã hết hiệu lực thì chỉ có 02 hình thức tài trợ đó là tiếp nhận tài trợ bằng tiền mặt hoặc tiếp nhận tài trợ bằng hiện vật. còn hiện nay tại khoản 3 điều 4 thông tư 16/2018/tt-bgdđt có thêm hình thức tài trợ phi vật chất" in p_tag.get_text().lower():
            p_tag.decompose()
        if p_tag.get_text().lower().endswith('.html') or p_tag.get_text().lower().endswith('.htm'):
            p_tag.decompose()
        img = p_tag.find('img')
        # Kiểm tra nếu thẻ img có thuộc tính style đúng như yêu cầu
        if img and img.get('style') == 'height:400px; width:600px':
            p_tag.decompose()  # Xóa thẻ <p> nếu điều kiện đúng
            
    # Tìm và xóa thẻ <img> có đuôi .gif
    for img_tag in soup.select('p img'):
        try:
            if re.search(r'\.gif$', img_tag['src']):
                img_tag.decompose()  # Xóa thẻ <img> khỏi cây HTML
        except Exception as e:
            print(f"Error processing tag: {e}")   
    # for p_tag in soup.find_all('p', style="text-align:center"):
    #     for em_tag in p_tag.find_all('em'):
    #         p_tag.decompose()
                    
    # xóa ảnh thumb và title ảnh
    # Lặp qua tất cả các thẻ <p> để kiểm tra điều kiện
    for i, p in enumerate(p_tags):
        # Kiểm tra nếu thẻ <p> chứa <img> và thẻ <p> tiếp theo có style "text-align: center"
        try:
            if p.find('img') and i + 1 < len(p_tags) and ('text-align: center' in p_tags[i + 1].get('style', '') or p_tags[i + 1].get_text().lower().endswith('internet)')):
                # or p_tags[i + 1].find('em')
                # Xóa cả hai thẻ <p> hiện tại và thẻ <p> tiếp theo
                p.decompose()
                p_tags[i + 1].decompose()
        except Exception as e:
            print(f"Error processing tag: {e}")
            
    # xóa nội dung bảng hết hiệu lực
    for table_tag in soup.find_all('s'):
        # print(table_tag.text)
        table_tag.decompose()
    
    # bỏ mấy bảng tham khảo màu đỏ
    for p_tag in soup.find_all('p'):
        for strong_tag in p_tag.find_all('strong', style="color: rgb(250, 10, 10);"):
            p_tag.decompose()
    
    # Xóa chữ "Xem thêm"
    for p_tag in soup.find_all('p'):
        if p_tag.get_text(strip=True).lower() == '>>xem thêm mẫu:' or p_tag.get_text(strip=True).lower() == "xem thêm:" or p_tag.get_text(strip=True).lower() == ">> xem thêm:":
            p_tag.decompose()
                   
    # Xóa chữ "Xem thêm"
    for p_tag in soup.find_all('p'):
        if p_tag.get_text(strip=True).lower() == '>>xem thêm mẫu' or p_tag.get_text(strip=True).lower() == "xem thêm" or p_tag.get_text(strip=True).lower() == ">> xem thêm":
            p_tag.decompose()
    
    # Xóa chữ "Về vấn đề này <span style="color:#FF0000;"><strong>THƯ VIỆN PHÁP LUẬT</strong></span> giải đáp như sau:</p>"
    for p_tag in soup.find_all('p'):
        for span_tag in p_tag.find_all('span'):
            if span_tag.get_text(strip=True).lower() == "thư viện pháp luật" or span_tag.get_text(strip=True).lower() == "pháp lý khởi nghiệp" or span_tag.get_text(strip=True).lower() == "pháp luật doanh nghiệp":
                p_tag.decompose()
                
    # Xóa chữ "trân trọng!"
    for p_tag in soup.find_all('p'):
        if p_tag.get_text(strip=True).lower() == "trân trọng!":
            p_tag.decompose()
        # Xóa chữ "trân trọng!"
    for p_tag in soup.find_all('p'):
        if p_tag.get_text(strip=True).lower() == "trân trọng":
            p_tag.decompose()
     # Xóa chữ "trân trọng!"
    for p_tag in soup.find_all('p'):
        if p_tag.get_text(strip=True).lower() == 'chúng tôi phản hồi thông tin đến bạn.':
            p_tag.decompose()
    # Xóa chữ "Trên đây là một số thông tin chúng tôi cung cấp gửi tới bạn. Trân trọng!"        
    for p_tag in soup.find_all('p'):
        if p_tag.get_text(strip=True).lower() == "trên đây là một số thông tin chúng tôi cung cấp gửi tới bạn. trân trọng!":
            p_tag.decompose()
    
    # xóa tên tác giả
    # <p style="text-align: right;"><strong>Như Mai</strong></p>
    for p_tag in soup.find_all('p', style="text-align: right;"):
        try:
            strong_tag = p_tag.find('strong')
            if strong_tag:  # Nếu tìm thấy thẻ <strong>
                p_tag.decompose()  # Xóa toàn bộ thẻ <p>
        except Exception as e:
            print(f"Error processing tag: {e}")
            
    for p_tag in soup.find_all('p', style="text-align: right;"):
        try:
            strong_tag = p_tag.find('em')
            if strong_tag:  # Nếu tìm thấy thẻ <strong>
                p_tag.decompose()  # Xóa toàn bộ thẻ <p>
        except Exception as e:
            print(f"Error processing tag: {e}")
    for p_tag in soup.find_all('p', align="right"):
        try:
            strong_tag = p_tag.find('strong')
            if strong_tag:  # Nếu tìm thấy thẻ <strong>
                p_tag.decompose()  # Xóa toàn bộ thẻ <p>
        except Exception as e:
            print(f"Error processing tag: {e}")
    for p_tag in soup.find_all('p', align="right"):
        try:
            strong_tag = p_tag.find('em')
            if strong_tag:  # Nếu tìm thấy thẻ <strong>
                p_tag.decompose()  # Xóa toàn bộ thẻ <p>
        except Exception as e:
            print(f"Error processing tag: {e}")
    
    # # chỉ duy nhất trang KN_FAQ và KN_JOB
    # # Tìm tất cả các thẻ table
    # tables = soup.find_all('table')

    # # Duyệt qua từng thẻ table
    # for table in tables:
    #     # Tìm tất cả các thẻ td trong table
    #     tds = table.find_all('td')
        
    #     # Nếu chỉ có duy nhất 1 thẻ td, thì xóa cả table
    #     if len(tds) == 1:
    #         table.decompose()
    # Xử lý phần tình huống pháp lý chứa trong nội dung
    # Tìm tất cả các thẻ <p> có chứa "Trả lời:"
    answer_paragraphs = soup.find_all('p', string=lambda text: text == "Trả lời:")
    # Lặp qua các thẻ <p> đã tìm được
    for answer_p in answer_paragraphs:
        # Tìm thẻ <h2> phía trên thẻ <p>
        h2_above = answer_p.find_previous('h2')
        
        if h2_above:
            # Lấy tất cả các phần tử từ <h2> tới <p> và xóa chúng (nhưng không xóa <h2>)
            current_element = h2_above.find_next_sibling()
            while current_element and current_element != answer_p:
                next_element = current_element.find_next_sibling()
                current_element.decompose()
                current_element = next_element
            # Xóa luôn thẻ <p> có chứa "Trả lời:"
            answer_p.decompose()
            
    # xóa các link dẫn
    for p_tag in soup.find_all('p'):
        # Lặp qua tất cả các thẻ <a> bên trong mỗi thẻ <p>
        for a_tag in p_tag.find_all('a', href=True):
            if a_tag is not None:  # Đảm bảo a_tag không phải là None
                try:
                    href_value = a_tag.get('href')  # Sử dụng get để tránh lỗi
                    if href_value and (href_value.endswith(('.html', '.htm')) or 'html?' in href_value):
                        # print(p_tag.text)
                        p_tag.decompose()  # Xóa thẻ <p> chứa thẻ <a> này
                    # extensions_end = ('.doc', '.docx', '.xls', '.xlsx', '.zip', '.rar', '.jpg', '.pdf', '.png')
                    extensions_start = ('>> xem chi tiết', 'xem chi tiết', '>> xem toàn bộ', 'xem toàn bộ', 'xem thêm', '>> xem thêm', 'xem chi tiết tại', '>> tham khảo', 'tham khảo', 'tải đầy đủ', '>>>')
                    if href_value and href_value.startswith('http') and (not '.aspx' in href_value) and p_tag.get_text(strip=True).lower().startswith(extensions_start):
                        p_tag.decompose()  # Xóa thẻ <p> chứa thẻ <a> này
                except AttributeError as e:
                    print(f"Đã gặp lỗi AttributeError: {e}")
    return soup

In [10]:
import os
import uuid
import base64

def save_base64_image(base64_string, file_extension="png", folder_name="img_base64_convert"):
    # Tạo thư mục nếu chưa tồn tại
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    
    # Giải mã chuỗi base64
    image_data = base64_string.split(',')[1]  # Tách phần dữ liệu base64
    image_bytes = base64.b64decode(image_data)

    # Tạo tên file ngẫu nhiên với phần mở rộng được chỉ định
    file_name = f"{uuid.uuid4()}.{file_extension}"

    # Đường dẫn đầy đủ để lưu file
    file_path = os.path.join(folder_name, file_name)

    # Lưu ảnh vào thư mục
    with open(file_path, 'wb') as f:
        f.write(image_bytes)

    return file_path


In [11]:

def processing_1(soup):
    questions_and_answers1 = []  # Danh sách để lưu trữ các link và anchor cho từng đoạn h2
    # Tìm tất cả các thẻ <h2>
    h2_tags = []
    for temp in soup.find_all('h2'):
        # Kiểm tra nếu thẻ <h2> có chứa thẻ <strong> và không phải là thẻ trống hoặc chỉ chứa <br>
        # if temp.find('strong') and temp.get_text(strip=True):
        if temp.get_text(strip=True):
            # and len(temp.get_text(strip=True)) >= 9:
            h2_tags.append(temp)
    # Lặp qua từng cặp thẻ <h2>
    # print(h2_tags)
    for i in range(len(h2_tags)):
        h2_current = h2_tags[i]
        if i + 1 < len(h2_tags):
            h2_next = h2_tags[i + 1]
        else:
            h2_next = 'Xuan Vinh Gay'

        patterns = r"^\d+\.\s*"
        # Tìm và thay thế phần (1) và phần 1. bằng chuỗi rỗng
        second_part = re.sub(patterns, "", h2_current.text).strip()
        second_part = unicodedata.normalize("NFC", second_part)
        # Lấy văn bản cho câu trả lời
        answer_text = ""
        next_element = h2_current.next_sibling
        while next_element and (not isinstance(next_element, Tag) or (next_element.name != 'h2')):
            if next_element and next_element.name == 'table':
                a_tag = next_element.find('a')
                # span_tag = next_element.find_all('span')
                if a_tag:
                    try:
                        href = a_tag['href']
                        # Danh sách các định dạng tài liệu
                        file_extensions = ('.doc', '.docx', '.xls', '.xlsx', '.zip', '.rar', '.jpg', '.pdf', '.png', '.jpeg')
                        # Kiểm tra nếu href kết thúc bằng một trong các định dạng tài liệu
                        if href.endswith(file_extensions):
                            # Tìm thẻ td chứa thẻ a
                            td_tag = a_tag.find_parent('td')
                            if td_tag:
                                # Tìm thẻ td kế tiếp (ngang hàng)
                                next_td = td_tag.find_next_sibling('td')
                                if next_td:
                                    link_text = next_td.get_text(strip=True)
                                    # Thay thế thẻ a bằng văn bản Markdown
                                    a_tag.replace_with(f"[{link_text}]({href})\n")
                                    answer_text += next_element.get_text() + "\n"
                        else:
                            # Nếu href không phải tài liệu
                            answer = unicodedata.normalize("NFC", next_element.get_text())
                            answer_text += answer + "\n"
                    except KeyError as e:
                        print(f"Thuộc tính 'href' không tồn tại trong thẻ <a>. Lỗi: {e}")
                else:
                    # Nếu không tìm thấy thẻ a
                    # print(table_to_markdown(next_element))
                    answer_text += table_to_markdown(next_element) + "\n"
            
            elif next_element and next_element.name == 'blockquote':
                if next_element.find('img'):
                    try:
                        img_tag = next_element.find('img')
                        src = img_tag['src']  
                        # Kiểm tra xem ảnh có phải là base64
                        if src.startswith('data:image'):
                            alt_text = img_tag['alt'] if img_tag.has_attr('alt') else ''
                            
                            # Lưu ảnh base64 vào file và chọn định dạng file (ví dụ: 'png')
                            file_name = save_base64_image(src, file_extension="png", folder_name="img_base64_convert")
                            
                            # Sử dụng tên file thay vì chuỗi base64
                            answer_text += f"![{alt_text}]({file_name})\n"
                        else:
                            alt_text = img_tag['alt'] if img_tag.has_attr('alt') else ''
                            answer_text += f"![{alt_text}]({src})\n"
                    except KeyError as e:
                        print(f"Thuộc tính 'src' không tồn tại. Lỗi: {e}")
                elif isinstance(next_element, NavigableString):
                    answer = unicodedata.normalize("NFC", next_element)
                    answer_text += answer.strip() + "\n"
                elif next_element and next_element.name:
                    answer = unicodedata.normalize("NFC", next_element.get_text())
                    answer_text += answer.strip() + "\n"
            elif next_element and next_element.name == 'p':
                if next_element.find('img'):
                    img_tag = next_element.find('img')
                    if img_tag:
                        try:
                            src = img_tag['src']

                            # Kiểm tra chuỗi base64
                            if src.startswith('data:image'):
                                alt_text = img_tag['alt'] if img_tag.has_attr('alt') else ''
                                
                                # Lưu ảnh base64 với định dạng file cố định (ví dụ: 'png')
                                file_name = save_base64_image(src, file_extension="png", folder_name="img_base64_convert")
                                
                                # Sử dụng tên file thay vì chuỗi base64
                                answer_text += f"![{alt_text}]({file_name})\n"
                            else:
                                alt_text = img_tag['alt'] if img_tag.has_attr('alt') else ''
                                answer_text += f"![{alt_text}]({src})\n"
                        except KeyError as e:
                            print(f"Thuộc tính 'src' không tồn tại trong thẻ <img>. Lỗi: {e}")            
                elif next_element.find('a'):
                    a_tag = next_element.find('a')
                    if a_tag:
                        try:
                            href = a_tag['href']
                            # Danh sách các định dạng tài liệu
                            file_extensions = ('.doc', '.docx', '.xls', '.xlsx', '.zip', '.rar', '.jpg', '.pdf', '.png', '.jpeg')
                            # Kiểm tra nếu href kết thúc bằng một trong các định dạng tài liệu
                            if href.endswith(file_extensions):
                                link_text = a_tag.text
                                for drop_a_tag in next_element.find('a'):
                                    drop_a_tag.replace_with(f"[{link_text}]({href})\n")
                                answer_text += next_element.get_text() + "\n"
                            else:
                                answer = unicodedata.normalize("NFC", next_element.get_text())
                                answer_text += answer.strip() + "\n"
                        except KeyError as e:
                            print(f"Thuộc tính 'href' không tồn tại trong <link>. Lỗi: {e}")
                # Kiểm tra chuỗi văn bản thuần
                elif isinstance(next_element, NavigableString):
                    answer = unicodedata.normalize("NFC", next_element)
                    answer_text += answer.strip() + "\n"
                
                # Kiểm tra thẻ khác và lấy văn bản
                elif next_element and next_element.name:
                    answer = unicodedata.normalize("NFC", next_element.get_text())
                    answer_text += answer.strip() + "\n"
            elif next_element and next_element.name == 'h3':
                patterns = r'^\d+(\.\d+)*\.\s*'
                if isinstance(next_element, NavigableString):
                    # Tìm và thay thế phần (1) và phần 1. bằng chuỗi rỗng
                    text_strip = re.sub(patterns, "", next_element).strip()
                    answer = unicodedata.normalize("NFC", text_strip)
                    answer_text += answer.strip() + "\n"
                elif next_element and next_element.name:
                    text_strip = re.sub(patterns, "", next_element.get_text()).strip()
                    answer = unicodedata.normalize("NFC", text_strip)
                    answer_text += answer.strip() + "\n"
            elif isinstance(next_element, NavigableString):
                    answer = unicodedata.normalize("NFC", next_element)
                    answer_text += answer.strip() + "\n"
            elif next_element and next_element.name:
                    answer = unicodedata.normalize("NFC", next_element.get_text())
                    answer_text += answer.strip() + "\n"
            next_element = next_element.next_sibling
        
        # Tìm tất cả các thẻ giữa h2_current và h2_next
        siblings = []
        next_tag = h2_current.find_next_sibling()
        while next_tag and next_tag != h2_next:
            siblings.append(next_tag)
            next_tag = next_tag.find_next_sibling()
        # Lặp qua các thẻ từ dưới lên trên trong các thẻ anh chị em và lưu kết quả tạm thời
        temp_results = []  # Danh sách tạm thời để lưu trữ các kết quả
        skip_next_p_check = True  # Biến cờ để kiểm tra thẻ <p> tiếp theo
        for tag in reversed(siblings):
            if tag.name == 'p':  # Kiểm tra nếu tag là thẻ <p>
                p_tag = tag.text
                # Kiểm tra xem nội dung của tag có bắt đầu bằng '(' và kết thúc bằng ')'
                if tag and tag.find('a') and p_tag.strip().startswith('Căn cứ pháp lý:'):
                    if skip_next_p_check:
                        for a_tag in tag.find_all('a'):
                            href = a_tag.get('href')
                            if href and '.aspx' in href and 'thuvienphapluat.vn' in href:
                                final = processing_ccpl_full(a_tag.text)
                                positions = split_by_dieu(final)
                                # test = p_tag.split(positions)
                                # print(positions)
                                if positions:
                                    lawtitle = positions[-1]
                                else:
                                    lawtitle = "default_value"  # Hoặc một giá trị mặc định nếu positions rỗng

                                # Kiểm tra nếu lawtitle không rỗng trước khi gọi split()
                                if lawtitle:
                                    test = p_tag.split(lawtitle)
                                else:
                                    # Xử lý khi lawtitle rỗng (nếu cần thiết)
                                    test = []
                                anchor_match = re.search(r'anchor=(\w+)', href)
                                anchor = anchor_match.group(1) if anchor_match else "0"
                                if test and "điều" in test[0].lower():
                                    p_tag = p_tag.replace(test[0], '', 1)
                                    final = processing_ccpl_full(test[0])
                                    positions = split_by_dieu(final)
                                    data_ccpl = []
                                    for dieu in positions:
                                            if "điều" in dieu.lower():
                                                dieu_dict = {"Dieu": extract_dieu(dieu), "Khoan": []}
                                                # Tìm anchor trong href
                                                khoan_list = split_by_khoan(dieu)
                                                for khoan in khoan_list:
                                                    if "khoản" in khoan.lower():
                                                        khoan_dict = {"Khoan": extract_khoan(khoan), "Diem": []}
                                                        if "điểm" in khoan.lower():
                                                            diem_dict = extract_diem(khoan)
                                                            khoan_dict["Diem"].append(diem_dict)
                                                        dieu_dict["Khoan"].append(khoan_dict)
                                                data_ccpl.append(dieu_dict)
                                                # print(data_ccpl)
                                                # Tạo một từ điển mới và gán giá trị từ mảng
                                                ccpl_temp = processing_output_ccpl(data_ccpl)
                                                if len(ccpl_temp) > 3:
                                                    for i in range(0, len(ccpl_temp), 3):
                                                        # Lấy 3 phần tử từ chỉ mục i đến i+3 (không bị lỗi nếu kích thước không đủ)
                                                        chunk = ccpl_temp[i:i+3]
                                                        anchor_match = re.search(r'anchor=(\w+)', href)
                                                        law_id_match = re.search(r'-(\d+)\.aspx', href)
                                                        anchor = anchor_match.group(1) if anchor_match else "0"
                                                        law_id = law_id_match.group(1) if law_id_match else "0"
                                                        # print(ccpl_temp)
                                                        if len(chunk) > 0:
                                                            dictionary = {
                                                                "url": href,
                                                                "anchor": anchor,
                                                                "LawID": law_id,
                                                                "LawTitle": lawtitle,
                                                                "Dieu": chunk[0] or "0",
                                                                "Khoan": chunk[1] or "0",
                                                                "Diem": chunk[2] or "0"
                                                            }
                                                            temp_results.append(dictionary)
                                                else:
                                                    anchor_match = re.search(r'anchor=(\w+)', href)
                                                    law_id_match = re.search(r'-(\d+)\.aspx', href)
                                                    anchor = anchor_match.group(1) if anchor_match else "0"
                                                    law_id = law_id_match.group(1) if law_id_match else "0"
                                                    if len(ccpl_temp) > 0:
                                                        dictionary = {
                                                            "url": href,
                                                            "anchor": anchor,
                                                            "LawID": law_id,
                                                            "LawTitle": lawtitle,
                                                            "Dieu": ccpl_temp[0] or "0",
                                                            "Khoan": ccpl_temp[1] or "0",
                                                            "Diem": ccpl_temp[2] or "0"
                                                        }
                                                        temp_results.append(dictionary)

                                else:
                                    temp_else = {}
                                    anchor_match = re.search(r'anchor=(\w+)', href)
                                    law_id_match = re.search(r'-(\d+)\.aspx', href)
                                    anchor = anchor_match.group(1) if anchor_match else "0"
                                    law_id = law_id_match.group(1) if law_id_match else "0"
                                    # law_title = a_tag.get_text()
                                    temp_else["url"] = href
                                    temp_else["anchor"] = anchor
                                    temp_else["LawID"] = law_id
                                    temp_else["LawTitle"] = lawtitle
                                    temp_else["Dieu"] = "0"
                                    temp_else["Khoan"] = "0"
                                    temp_else["Diem"] = "0"
                                    temp_results.append(temp_else)
                    skip_next_p_check = False
        ccpls = []
        ccpls.extend(temp_results)
        
        questions_and_answers1.append({
            "question": second_part,
            "answer": answer_text.strip(),
            "ccpls": ccpls
        })
    patterns = [    
        re.compile(r"Chọn lĩnh vực để xem văn bản liên quan.*", re.MULTILINE),
        re.compile(r">> Quý khách hàng xem thêm.*", re.MULTILINE),
        re.compile(r"Quý khách hàng xem thêm.*", re.MULTILINE),
        re.compile(r"Quý khách hành xem thêm.*", re.MULTILINE),    
        re.compile(r"Quý khách hàng có thể.*", re.MULTILINE),
    ]

    for item in questions_and_answers1:
        answer_text = item["answer"]
        # Khởi tạo start_index với giá trị -1 (không tìm thấy)
        start_index = -1
        # Kiểm tra từng mẫu regex và cập nhật start_index nếu tìm thấy
        for pattern in patterns:
            match = pattern.search(answer_text)
            if match:
                index = match.start()
                # Nếu start_index chưa được cập nhật hoặc tìm thấy index nhỏ hơn (gần đầu chuỗi hơn)
                if start_index == -1 or index < start_index:
                    start_index = index

        # Nếu tìm thấy bất kỳ mẫu nào, xóa từ đoạn đó đến hết câu trả lời
        if start_index != -1:
            cleaned_answer = answer_text[:start_index].strip()
            item["answer"] = cleaned_answer
    # print(json.dumps(questions_and_answers1, ensure_ascii=False, indent=5))        
    # # In ra danh sách câu hỏi và câu trả lời đã được làm sạch
    return json.dumps(questions_and_answers1, ensure_ascii=False, indent=5)


In [12]:
def processing_5(soup):
    h2_tags = []
    for temp in soup.find_all('h2'):
        # Kiểm tra nếu thẻ <h2> có chứa thẻ <strong> và không phải là thẻ trống hoặc chỉ chứa <br>
        # if temp.find('strong') and temp.get_text(strip=True):
        if temp.get_text(strip=True):
        # and len(temp.get_text(strip=True)) >= 9:
            h2_tags.append(temp)

    questions_and_answers5 = []  # Danh sách để lưu trữ các link và anchor cho từng đoạn h2


    for h2_tag in h2_tags:
        temp_results = []
        next_tag = h2_tag.find_next_sibling()
        gay_flag = True
        while next_tag and next_tag.name != 'h2':
            if next_tag.name == 'p':
                p_tag = next_tag.text
                if (re.search(r"thư\s+viện\s+pháp\s+luật", next_tag.text.lower())):
                    gay_flag = False
                if next_tag.find('a') and not next_tag.text.startswith('>>') and gay_flag == True:
                    for a_tag in next_tag.find_all('a'):
                        href = a_tag.get('href')
                        if href and '.aspx' in href and 'thuvienphapluat.vn' in href:
                            final = processing_ccpl_full(a_tag.text)
                            positions = split_by_dieu(final)
                            # test = p_tag.split(positions)
                            # print(positions)
                            if positions:
                                lawtitle = positions[-1]
                            else:
                                lawtitle = "default_value"  # Hoặc một giá trị mặc định nếu positions rỗng

                            # Kiểm tra nếu lawtitle không rỗng trước khi gọi split()
                            if lawtitle:
                                test = p_tag.split(lawtitle)
                            else:
                                # Xử lý khi lawtitle rỗng (nếu cần thiết)
                                test = []
                            # print(lawtitle)
                            anchor_match = re.search(r'anchor=(\w+)', href)
                            anchor = anchor_match.group(1) if anchor_match else "0"
                            if test and "điều" in test[0].lower():
                                p_tag = p_tag.replace(test[0], '', 1)
                                final = processing_ccpl_full(test[0])
                                positions = split_by_dieu(final)
                                data_ccpl = []
                                for dieu in positions:
                                        if "điều" in dieu.lower():
                                            dieu_dict = {"Dieu": extract_dieu(dieu), "Khoan": []}
                                            # Tìm anchor trong href
                                            khoan_list = split_by_khoan(dieu)
                                            for khoan in khoan_list:
                                                if "khoản" in khoan.lower():
                                                    khoan_dict = {"Khoan": extract_khoan(khoan), "Diem": []}
                                                    if "điểm" in khoan.lower():
                                                        diem_dict = extract_diem(khoan)
                                                        khoan_dict["Diem"].append(diem_dict)
                                                    dieu_dict["Khoan"].append(khoan_dict)
                                            data_ccpl.append(dieu_dict)
                                            # print(data_ccpl)
                                            # Tạo một từ điển mới và gán giá trị từ mảng
                                            ccpl_temp = processing_output_ccpl(data_ccpl)
                                            if len(ccpl_temp) > 3:
                                                for i in range(0, len(ccpl_temp), 3):
                                                    # Lấy 3 phần tử từ chỉ mục i đến i+3 (không bị lỗi nếu kích thước không đủ)
                                                    chunk = ccpl_temp[i:i+3]
                                                    anchor_match = re.search(r'anchor=(\w+)', href)
                                                    law_id_match = re.search(r'-(\d+)\.aspx', href)
                                                    anchor = anchor_match.group(1) if anchor_match else "0"
                                                    law_id = law_id_match.group(1) if law_id_match else "0"
                                                    # print(ccpl_temp)
                                                    if len(chunk) > 0:
                                                        dictionary = {
                                                            "url": href,
                                                            "anchor": anchor,
                                                            "LawID": law_id,
                                                            "LawTitle": lawtitle,
                                                            "Dieu": chunk[0] or "0",
                                                            "Khoan": chunk[1] or "0",
                                                            "Diem": chunk[2] or "0"
                                                        }
                                                        temp_results.append(dictionary)
                                            else:
                                                anchor_match = re.search(r'anchor=(\w+)', href)
                                                law_id_match = re.search(r'-(\d+)\.aspx', href)
                                                anchor = anchor_match.group(1) if anchor_match else "0"
                                                law_id = law_id_match.group(1) if law_id_match else "0"
                                                if len(ccpl_temp) > 0:
                                                    dictionary = {
                                                        "url": href,
                                                        "anchor": anchor,
                                                        "LawID": law_id,
                                                        "LawTitle": lawtitle,
                                                        "Dieu": ccpl_temp[0] or "0",
                                                        "Khoan": ccpl_temp[1] or "0",
                                                        "Diem": ccpl_temp[2] or "0"
                                                    }
                                                    temp_results.append(dictionary)

                            else:
                                temp_else = {}
                                anchor_match = re.search(r'anchor=(\w+)', href)
                                law_id_match = re.search(r'-(\d+)\.aspx', href)
                                anchor = anchor_match.group(1) if anchor_match else "0"
                                law_id = law_id_match.group(1) if law_id_match else "0"
                                # law_title = a_tag.get_text()
                                temp_else["url"] = href
                                temp_else["anchor"] = anchor
                                temp_else["LawID"] = law_id
                                temp_else["LawTitle"] = lawtitle
                                temp_else["Dieu"] = "0"
                                temp_else["Khoan"] = "0"
                                temp_else["Diem"] = "0"
                                temp_results.append(temp_else)
                    gay_flag = True
            next_tag = next_tag.find_next_sibling()
        ccpls = []
        # ccpls.extend(temp_results)
        ccpls.extend(temp_results)
        questions_and_answers5.append({
            "ccpls": ccpls
        })
    # In ra danh sách câu hỏi và câu trả lời đã được làm sạch
    return json.dumps(questions_and_answers5, ensure_ascii=False, indent=5)
# print(json.dumps(questions_and_answers4, ensure_ascii=False, indent=5))

In [13]:
# hàm giải mã lawid
def decrypt(s):
    try:
        tem = s
        if len(tem) > 4:
            tem = tem[-2:] + tem[2:-2] + tem[:2]
        return base64.b64decode(base64.b64decode(tem).decode('utf-8')).decode('utf-8')
    except:
        return ""

def decode_ccpls(questions_and_answers_final):
    for temp in questions_and_answers_final:
        for temp1 in temp.get('ccpls'):
            if '.aspx?id=' in temp1.get('url'):
                # Phân tích cú pháp URL
                parsed_url = urlparse(temp1.get('url'))
                # Lấy các tham số truy vấn từ URL
                query_params = parse_qs(parsed_url.query)
                # Lấy giá trị của tham số 'id'
                id_value = query_params.get('id', [None])[0]
                temp1['LawID'] = decrypt(id_value)
    return questions_and_answers_final

In [14]:
def extract_data(data):
    soup = processing_redundant_data(data)
    file1_data = json.loads(processing_1(soup))
    file5_data = json.loads(processing_5(soup))
    
    # ghép ccpl loại 5
    for index, item in enumerate(file1_data):
        if not item['ccpls']:
            item['ccpls'] = file5_data[index]['ccpls']
    
    # Chỉ giữ lại các mục có câu trả lời
    filtered_questions_and_answers = [qa for qa in file1_data if 'answer' in qa and qa['answer'].strip()]
    filtered_questions_and_answers1 = [qa for qa in filtered_questions_and_answers if 'question' in qa and qa['question'].strip()]
    
    questions_and_answers_final = []
    for qa in filtered_questions_and_answers1:
        question_text = qa["question"]
        # Kiểm tra xem câu hỏi có chứa các từ "dự thảo", "đề xuất", "sắp tới" hoặc "dự kiến" không
        if not re.search(r'\b(?:dự thảo|đề xuất|sắp tới|dự kiến|đáp án|tra cứu điểm thi)\b', question_text.lower(), flags=re.IGNORECASE) and not question_text.lower().startswith("đã có"):
            questions_and_answers_final.append(qa)
    # Remove duplicates
    filtered_questions_and_answers1 = remove_duplicates_ccpls(filtered_questions_and_answers1)
    filtered_questions_and_answers1  = decode_ccpls(filtered_questions_and_answers1)
    return filtered_questions_and_answers1
    # return json.dumps(questions_and_answers_final, ensure_ascii=False)

function html request

In [15]:
import requests
import json
import html
import re
from bs4 import BeautifulSoup

def post_request(url, headers):
    response = requests.post(url, headers=headers)
    return response.json()


def post_request_key(url, headers, data):
    try:
        data = requests.post(url, headers=headers, data=json.dumps(data))
        data.raise_for_status()  # Raise an error for bad status codes
        return data.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None


get api

In [16]:
# Sử dụng hàm để thực hiện yêu cầu POST
def get_api():
    url1 = 'https://apids.thuvienphapluat.vn/auth/get-token?key=pvaG4gRG9lIiwiaWF0IjoxNTE2MjM5MDIyfQS563xADXH'
    headers1 = {
        'Cookie': 'Culture=vi; Culture=vi'
    }

    key = post_request(url1, headers1)

    url = 'https://apids.thuvienphapluat.vn/data/get-phapluat'
    headers = {
        'Authorization': 'Bearer ' + key['Data']['AccessToken'],
        'Content-Type': 'application/json',
        'Cookie': 'Culture=vi'
    }
    return url, headers

def send_data_to_api(data):
    url, headers = get_api()
    response = requests.post(url, headers=headers, json=data)
    return response.json()

POST data

In [17]:

url = 'https://thuvienphapluat.vn/phap-luat/chung-chi-dai-ly-bao-hiem-gom-nhung-chung-chi-nao-cap-chung-chi-dai-ly-bao-hiem-khong-dung-quy-dinh-962551-169772.html#cap-chung-chi-dai-ly-bao-hiem-khong-dung-quy-dinh-thi-co-so-dao-tao-dai-ly-bi-xu-phat-the-nao-2'
# Gửi yêu cầu HTTP để lấy nội dung của trang web
response = requests.get(url)

# Kiểm tra xem yêu cầu có thành công không (mã trạng thái 200 là thành công)
if response.status_code == 200:
    # Lấy nội dung của trang web
    html_content = response.text
    # print(html_content)
    print(json.dumps(extract_data(html_content), ensure_ascii=False))

Error processing tag: 'NoneType' object has no attribute 'find'
Error processing tag: 'NoneType' object has no attribute 'find'
[{"question": "Thời hạn nộp tiền thuế TNDN tạm tính quý 2 năm 2024?", "answer": "Căn cứ Khoản 1 Điều 55 Luật Quản lý Thuế 2019 quy định về thời hạn nộp thuế như sau:\nThời hạn nộp thuế\n1. Trường hợp người nộp thuế tính thuế, thời hạn nộp thuế chậm nhất là ngày cuối cùng của thời hạn nộp hồ sơ khai thuế. Trường hợp khai bổ sung hồ sơ khai thuế, thời hạn nộp thuế là thời hạn nộp hồ sơ khai thuế của kỳ tính thuế có sai, sót.\nĐối với thuế thu nhập doanh nghiệp thì tạm nộp theo quý, thời hạn nộp thuế chậm nhất là ngày 30 của tháng đầu quý sau.\n…\nNhư vậy, thời hạn nộp thuế TNDN tạm tính quý 2 (tức tháng 4,5,6) chậm nhất là ngày 30 của tháng đầu quý 3 tức ngày 30/07/2024. Trường hợp ngày cuối cùng của thời hạn là ngày nghỉ cuối tuần hoặc ngày nghỉ lễ thì thời hạn kết thúc tại thời điểm kết thúc ngày làm việc tiếp theo ngày nghỉ đó theo quy định tại khoản 5 Điều 1

In [18]:
data_api = {
    "page": 1, #14039
    "num": 1,
    "field":0,
    "expireccpl": 2,
    "type": 2
}
file_note = []
data_type1 = []
data_type7 = []
data_type3 = []
url, headers = get_api()
response = post_request_key(url, headers, data_api)

if response:
    specific_page = response.get('total')
    print(specific_page)
now = ((136127)//100)+1
num = 100
total_page = specific_page // num + ( 1 if specific_page % num > 0 else 0)
for num_page in range(now, total_page + 1):
# for num_page in range(total_page, 0, - 1):
    data_api = {
        "page": num_page, #14039
        "num": num,
        "field":0,
        "expireccpl": 2,
        "type": 2
    }
    url1 = 'https://apids.thuvienphapluat.vn/auth/get-token?key=pvaG4gRG9lIiwiaWF0IjoxNTE2MjM5MDIyfQS563xADXH'
    headers1 = {
        'Cookie': 'Culture=vi; Culture=vi'
    }

    key = post_request(url1, headers1)
    url_post = "https://apids.thuvienphapluat.vn/crud/log-extracted-news-data"

    headers_post = {
    'Authorization': 'Bearer ' + key['Data']['AccessToken'],
    'Content-Type': 'application/json',
    'Cookie': 'Culture=vi'
    }

    url, headers = get_api()
    response = post_request_key(url, headers, data_api)
    # print(response.get('data'))
    data_input = response.get('data')
    for temp in data_input:
        # print(temp)
        # print(temp["content"])
        # print(temp['title'])
        extracted_data = extract_data(temp["content"])
        # print(extract_data(temp["content"]))
        # ccpls_flags = True
        if extracted_data != [] and not re.search(r'\b(?:dự thảo|đề xuất|sắp tới|dự kiến|đáp án|tra cứu điểm thi)\b', temp["title"].lower(), flags=re.IGNORECASE):
            # for ccpl_check in extracted_data:
            #     if not ccpl_check.get('ccpls'):
            #         ccpls_flags = False
            #         break
            # if not ccpls_flags:    
            #     dictionary = {
            #         "objid": int(temp['obj_id']),
            #         "source": str(temp['obj_code']),
            #         "data": json.dumps(extracted_data, ensure_ascii=False),
            #             # extracted_data,
            #         "type": 7
            #     }
            #     dictionary = json.dumps(dictionary, ensure_ascii=False)
            #     data_type7.append(int(temp['obj_id']))
            #     response = requests.request("POST", url_post, headers=headers_post, data=dictionary)
            #     print(response.text)
            # else:
            #     # for temp_extract_data in extracted_data:
            #     #     if not temp_extract_data.get('ccpls'):
            #     #         temp_extract_data['ccpls'] = result_ccpl(temp_extract_data.get('answer'), get_items())
            dictionary = {
                "objid": int(temp['obj_id']),
                "source": str(temp['obj_code']),
                "data": json.dumps(extracted_data, ensure_ascii=False),
                    # extracted_data,
                "type": 1
            }
            dictionary = json.dumps(dictionary, ensure_ascii=False)
            data_type1.append(int(temp['obj_id']))
            response = requests.request("POST", url_post, headers=headers_post, data=dictionary)
            print(response.text)
        elif extracted_data != [] and re.search(r'\b(?:dự thảo|đề xuất|sắp tới|dự kiến|đáp án|tra cứu điểm thi)\b', temp["title"].lower(), flags=re.IGNORECASE):
            dictionary = {
                "objid": int(temp['obj_id']),
                "source": str(temp['obj_code']),
                "data": json.dumps(extracted_data, ensure_ascii=False),
                "type": 3
            }
            dictionary = json.dumps(dictionary, ensure_ascii=False)
            data_type3.append(int(temp['obj_id']))
            response = requests.request("POST", url_post, headers=headers_post, data=dictionary)
            print(response.text)
        else:
            file_note.append(int(temp['obj_id']))
            # print(dictionary)
            # time.sleep(1)
now = specific_page
with open('PL_type1_l3.txt', 'w') as file:
    for item in data_type1:
        file.write(f"{item}\n")
with open('PL_type3_l3.txt', 'w') as file:
    for item in data_type3:
        file.write(f"{item}\n")
with open('PL_type7_l3.txt', 'w') as file:
    for item in data_type7:
        file.write(f"{item}\n")
with open('PL_file_note_l3.txt', 'w') as file:
    for item in file_note:
        file.write(f"{item}\n")

136283
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
Erro

In [19]:
with open('PL_type1_l3.txt', 'w') as file:
    for item in data_type1:
        file.write(f"{item}\n")
with open('PL_type3_l3.txt', 'w') as file:
    for item in data_type3:
        file.write(f"{item}\n")
with open('PL_type7_l3.txt', 'w') as file:
    for item in data_type7:
        file.write(f"{item}\n")
with open('PL_file_note_l3.txt', 'w') as file:
    for item in file_note:
        file.write(f"{item}\n")